Agente ReACT con LangGraph Components

In [2]:
from langgraph.graph import StateGraph
from typing import TypedDict, List
from groq import Groq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
import os
import re

c:\Users\Fabi\Documents\repos\ceia-llm\venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
class AgentState(TypedDict):
    question: str
    context: List[str]
    detector: List[str]
    agent: str
    llm: str
    complete: bool

In [4]:
class AgentRag:
    
    def __init__(self, index): 

        # Embeddings
        self.embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
        
        # Conexión con base de datos
        self.vstore = PineconeVectorStore(
            embedding=self.embed_model,
            index_name=index,
            pinecone_api_key=os.getenv("API_KEY_PINECONE")
        )
       
    def search(self, state: AgentState):
        if "context" in state:
            context_list = state["context"]
        else:
            context_list = []
        
        context = ""
        results = self.vstore.similarity_search(state["question"], k=3)
        for res in results:
            context += res.page_content
        
        return { "context": context_list + [context] } 

In [5]:
agent_1 = AgentRag("fabian-cv")
agent_2 = AgentRag("mock-cv")

In [10]:
agent_1.search({"question": "de que trabaja Fabian?"})

{'context': ['key to success. A problem solver at heart, I approach\nchallenging scenarios with a creative mindset and enjoy\nfinding solutions.\nPROFILE\nCountry of origin: Argentina\nEmail: massotto.fabian@gmail.com\nPhone: +54 9 11 3146 2971\nLinkedIn: linkedin.com/in/massottofabian\nGitHub: github.com/fabimass\nPortfolio: fabianmassotto.vercel.app\nCONTACT INFO\nPython\nDjango\nAzure\nTerraform\nGitHub\nDocker\nBackstage\nStorybookboost productivity for engineering teams.\nErnst & Young\nSoftware Engineer\nJun 2019 - Jul 2020\nAs a member of the Computer Forensics team, my main\nresponsibilities involved engineering ETL pipelines to\ntransform digital evidence into structured formats, making\ndata accessible and actionable for auditors and fraud\ninvestigators.\nCAREER SUMMARY\nUniversity of Buenos Aires\nMaster’s degree, Artificial Intelligence         Mar 2024  - Present\nUniversidad Tecnológica NacionalFABIAN MASSOTTO\nSOFTWARE ENGINEER\nExxonMobil\nSoftware Engineer\nJul 2020 -

In [12]:
agent_2.search({"question": "de que trabaja Richard?"})

{'context': ["EDUCATION\nRICHARD SANCHEZ\nMARKETING MANAGER \nCONTACT\n+123-456-7890\nhello@reallygreatsite.com\n123 Anywhere St., Any City\nwww.reallygreatsite.com\nSKILLS\nProject Management\nPublic Relations\nTeamwork\nTime Management\nLeadership\nEffective Communication\nCritical Thinking\nWARDIERE UNIVERSITY\nMaster of Business\nManagement\n2029 - 2030\n2025 - 2029 \nWARDIERE UNIVERSITY\nBachelor of Business\nGPA: 3.8 / 4.0\nEnglish (Fluent)\nFrench (Fluent)\nGerman (Basics)\nSpanish (Intermediate)\nLANGUAGES\nWORK EXPERIENCE\nREFERENCE\nPROFILELANGUAGES\nWORK EXPERIENCE\nREFERENCE\nPROFILE\nLorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor\nincididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis\nnostrud exercitation. Lorem ipsum dolor sit amet, consectetur adipiscing elit,\nsed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad\nminim veniam quis nostrud exercitation. Ut enim ad minim veniam quis nostrud\nexer

In [6]:
# Crea el cliente de GROQ
llm_client = Groq(api_key=os.environ.get("API_KEY_GROQ"))

def llm(state: AgentState):
    input = state["question"]
    context = ""
    for ctx in state["context"]:
        context += ctx
        
    query = f"""
    Contesta la siguiente pregunta en base al contexto provisto.

    Pregunta: {input}

    Contexto: {context}
    """

    # Genera la respuesta del chatbot utilizando el modelo LLaMA 3
    chat_completion = llm_client.chat.completions.create(
        messages=[{"role": "user", "content": query}],
        model="llama3-8b-8192",
    )
    response = chat_completion.choices[0].message.content

    return {"llm": response}

In [7]:
# Esta funcion detecta patrones en la pregunta del usuario
def detector(state: AgentState):
    persons_detected = []
    patterns = {
        "mock": r'\brichard\b.*\?',
        "fabian": r'\bfabian\b.*\?'
    }
    
    for name, pattern in patterns.items():
        match = bool(re.search(pattern, state["question"], re.IGNORECASE))
        if match:
            persons_detected.append(name)

    # Si no se detecto ningun patron, se va a utilizar el cv del alumno
    if len(persons_detected) == 0:
        persons_detected.append("fabian")
        
    return {"detector": persons_detected, "context": []}

In [8]:
# Esta funcion chequea si tiene que seguir iterando o no
def orchestrator(state: AgentState):
    print(state)
    if len(state["context"]) == len(state["detector"]):
        return { "complete": True } 
    else:
        return { "complete": False } 

In [9]:
# Esta funcion indica a que nodo ir desde el orchestrator
def next_node(state: AgentState):
    if state["complete"]:
        return "llm"
    else:
        return state["detector"][len(state["context"])]

In [10]:
builder = StateGraph(AgentState)

builder.add_node("detector_node", detector)
builder.add_node("orchestrator_node", orchestrator)
builder.add_node("agent_1", agent_1.search)
builder.add_node("agent_2", agent_2.search)
builder.add_node("llm_node", llm)

builder.add_edge("detector_node", "orchestrator_node")
builder.add_conditional_edges(
    "orchestrator_node", 
    next_node, 
    {"fabian": "agent_1", "mock": "agent_2", "llm": "llm_node"}
)
builder.add_edge("agent_1", "orchestrator_node")
builder.add_edge("agent_2", "orchestrator_node")

builder.set_entry_point("detector_node")

graph = builder.compile()

In [13]:
print(graph.get_graph().draw_mermaid())

%%{init: {'flowchart': {'curve': 'linear'}}}%%
graph TD;
	__start__([<p>__start__</p>]):::first
	detector_node(detector_node)
	orchestrator_node(orchestrator_node)
	agent_1(agent_1)
	agent_2(agent_2)
	llm_node([llm_node]):::last
	__start__ --> detector_node;
	agent_1 --> orchestrator_node;
	agent_2 --> orchestrator_node;
	detector_node --> orchestrator_node;
	orchestrator_node -. &nbsp;fabian&nbsp; .-> agent_1;
	orchestrator_node -. &nbsp;mock&nbsp; .-> agent_2;
	orchestrator_node -. &nbsp;llm&nbsp; .-> llm_node;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



![graph](./graph.png)

In [63]:
graph.invoke({"question": "cual es la profesion de Fabian?"})

{'question': 'cual es la profesion de Fabian?', 'context': [], 'detector': ['fabian']}
{'question': 'cual es la profesion de Fabian?', 'context': ['key to success. A problem solver at heart, I approach\nchallenging scenarios with a creative mindset and enjoy\nfinding solutions.\nPROFILE\nCountry of origin: Argentina\nEmail: massotto.fabian@gmail.com\nPhone: +54 9 11 3146 2971\nLinkedIn: linkedin.com/in/massottofabian\nGitHub: github.com/fabimass\nPortfolio: fabianmassotto.vercel.app\nCONTACT INFO\nPython\nDjango\nAzure\nTerraform\nGitHub\nDocker\nBackstage\nStorybookboost productivity for engineering teams.\nErnst & Young\nSoftware Engineer\nJun 2019 - Jul 2020\nAs a member of the Computer Forensics team, my main\nresponsibilities involved engineering ETL pipelines to\ntransform digital evidence into structured formats, making\ndata accessible and actionable for auditors and fraud\ninvestigators.\nCAREER SUMMARY\nUniversity of Buenos Aires\nMaster’s degree, Artificial Intelligence     

{'question': 'cual es la profesion de Fabian?',
 'context': ['key to success. A problem solver at heart, I approach\nchallenging scenarios with a creative mindset and enjoy\nfinding solutions.\nPROFILE\nCountry of origin: Argentina\nEmail: massotto.fabian@gmail.com\nPhone: +54 9 11 3146 2971\nLinkedIn: linkedin.com/in/massottofabian\nGitHub: github.com/fabimass\nPortfolio: fabianmassotto.vercel.app\nCONTACT INFO\nPython\nDjango\nAzure\nTerraform\nGitHub\nDocker\nBackstage\nStorybookboost productivity for engineering teams.\nErnst & Young\nSoftware Engineer\nJun 2019 - Jul 2020\nAs a member of the Computer Forensics team, my main\nresponsibilities involved engineering ETL pipelines to\ntransform digital evidence into structured formats, making\ndata accessible and actionable for auditors and fraud\ninvestigators.\nCAREER SUMMARY\nUniversity of Buenos Aires\nMaster’s degree, Artificial Intelligence         Mar 2024  - Present\nUniversidad Tecnológica NacionalFABIAN MASSOTTO\nSOFTWARE ENG

In [61]:
graph.invoke({"question": "cual es la profesion de Richard?"})

{'question': 'cual es la profesion de Richard?', 'context': [], 'detector': ['mock']}
{'question': 'cual es la profesion de Richard?', 'context': ["EDUCATION\nRICHARD SANCHEZ\nMARKETING MANAGER \nCONTACT\n+123-456-7890\nhello@reallygreatsite.com\n123 Anywhere St., Any City\nwww.reallygreatsite.com\nSKILLS\nProject Management\nPublic Relations\nTeamwork\nTime Management\nLeadership\nEffective Communication\nCritical Thinking\nWARDIERE UNIVERSITY\nMaster of Business\nManagement\n2029 - 2030\n2025 - 2029 \nWARDIERE UNIVERSITY\nBachelor of Business\nGPA: 3.8 / 4.0\nEnglish (Fluent)\nFrench (Fluent)\nGerman (Basics)\nSpanish (Intermediate)\nLANGUAGES\nWORK EXPERIENCE\nREFERENCE\nPROFILELANGUAGES\nWORK EXPERIENCE\nREFERENCE\nPROFILE\nLorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor\nincididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis\nnostrud exercitation. Lorem ipsum dolor sit amet, consectetur adipiscing elit,\nsed do eiusmod tempor incid

{'question': 'cual es la profesion de Richard?',
 'context': ["EDUCATION\nRICHARD SANCHEZ\nMARKETING MANAGER \nCONTACT\n+123-456-7890\nhello@reallygreatsite.com\n123 Anywhere St., Any City\nwww.reallygreatsite.com\nSKILLS\nProject Management\nPublic Relations\nTeamwork\nTime Management\nLeadership\nEffective Communication\nCritical Thinking\nWARDIERE UNIVERSITY\nMaster of Business\nManagement\n2029 - 2030\n2025 - 2029 \nWARDIERE UNIVERSITY\nBachelor of Business\nGPA: 3.8 / 4.0\nEnglish (Fluent)\nFrench (Fluent)\nGerman (Basics)\nSpanish (Intermediate)\nLANGUAGES\nWORK EXPERIENCE\nREFERENCE\nPROFILELANGUAGES\nWORK EXPERIENCE\nREFERENCE\nPROFILE\nLorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor\nincididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis\nnostrud exercitation. Lorem ipsum dolor sit amet, consectetur adipiscing elit,\nsed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad\nminim veniam quis nostrud exercit

In [58]:
graph.invoke({"question": "donde trabaja?"})

{'question': 'donde trabaja?', 'context': [], 'detector': ['fabian']}
{'question': 'donde trabaja?', 'context': ['key to success. A problem solver at heart, I approach\nchallenging scenarios with a creative mindset and enjoy\nfinding solutions.\nPROFILE\nCountry of origin: Argentina\nEmail: massotto.fabian@gmail.com\nPhone: +54 9 11 3146 2971\nLinkedIn: linkedin.com/in/massottofabian\nGitHub: github.com/fabimass\nPortfolio: fabianmassotto.vercel.app\nCONTACT INFO\nPython\nDjango\nAzure\nTerraform\nGitHub\nDocker\nBackstage\nStorybookboost productivity for engineering teams.\nErnst & Young\nSoftware Engineer\nJun 2019 - Jul 2020\nAs a member of the Computer Forensics team, my main\nresponsibilities involved engineering ETL pipelines to\ntransform digital evidence into structured formats, making\ndata accessible and actionable for auditors and fraud\ninvestigators.\nCAREER SUMMARY\nUniversity of Buenos Aires\nMaster’s degree, Artificial Intelligence         Mar 2024  - Present\nUniversid

{'question': 'donde trabaja?',
 'context': ['key to success. A problem solver at heart, I approach\nchallenging scenarios with a creative mindset and enjoy\nfinding solutions.\nPROFILE\nCountry of origin: Argentina\nEmail: massotto.fabian@gmail.com\nPhone: +54 9 11 3146 2971\nLinkedIn: linkedin.com/in/massottofabian\nGitHub: github.com/fabimass\nPortfolio: fabianmassotto.vercel.app\nCONTACT INFO\nPython\nDjango\nAzure\nTerraform\nGitHub\nDocker\nBackstage\nStorybookboost productivity for engineering teams.\nErnst & Young\nSoftware Engineer\nJun 2019 - Jul 2020\nAs a member of the Computer Forensics team, my main\nresponsibilities involved engineering ETL pipelines to\ntransform digital evidence into structured formats, making\ndata accessible and actionable for auditors and fraud\ninvestigators.\nCAREER SUMMARY\nUniversity of Buenos Aires\nMaster’s degree, Artificial Intelligence         Mar 2024  - Present\nUniversidad Tecnológica NacionalFABIAN MASSOTTO\nSOFTWARE ENGINEER\nExxonMobil

In [64]:
graph.invoke({"question": "cual es la diferencia entre la profesion de Fabian y la de Richard?"})

{'question': 'cual es la diferencia entre la profesion de Fabian y la de Richard?', 'context': [], 'detector': ['mock', 'fabian']}
{'question': 'cual es la diferencia entre la profesion de Fabian y la de Richard?', 'context': ["EDUCATION\nRICHARD SANCHEZ\nMARKETING MANAGER \nCONTACT\n+123-456-7890\nhello@reallygreatsite.com\n123 Anywhere St., Any City\nwww.reallygreatsite.com\nSKILLS\nProject Management\nPublic Relations\nTeamwork\nTime Management\nLeadership\nEffective Communication\nCritical Thinking\nWARDIERE UNIVERSITY\nMaster of Business\nManagement\n2029 - 2030\n2025 - 2029 \nWARDIERE UNIVERSITY\nBachelor of Business\nGPA: 3.8 / 4.0\nEnglish (Fluent)\nFrench (Fluent)\nGerman (Basics)\nSpanish (Intermediate)\nLANGUAGES\nWORK EXPERIENCE\nREFERENCE\nPROFILELANGUAGES\nWORK EXPERIENCE\nREFERENCE\nPROFILE\nLorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor\nincididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis\nnostrud exercitation. Lorem

{'question': 'cual es la diferencia entre la profesion de Fabian y la de Richard?',
 'context': ["EDUCATION\nRICHARD SANCHEZ\nMARKETING MANAGER \nCONTACT\n+123-456-7890\nhello@reallygreatsite.com\n123 Anywhere St., Any City\nwww.reallygreatsite.com\nSKILLS\nProject Management\nPublic Relations\nTeamwork\nTime Management\nLeadership\nEffective Communication\nCritical Thinking\nWARDIERE UNIVERSITY\nMaster of Business\nManagement\n2029 - 2030\n2025 - 2029 \nWARDIERE UNIVERSITY\nBachelor of Business\nGPA: 3.8 / 4.0\nEnglish (Fluent)\nFrench (Fluent)\nGerman (Basics)\nSpanish (Intermediate)\nLANGUAGES\nWORK EXPERIENCE\nREFERENCE\nPROFILELANGUAGES\nWORK EXPERIENCE\nREFERENCE\nPROFILE\nLorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor\nincididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis\nnostrud exercitation. Lorem ipsum dolor sit amet, consectetur adipiscing elit,\nsed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad